In [51]:
import cv2
import numpy as np
import dlib
import pyautogui as gui

In [52]:
## 인식기 load 및 상수 파라미터 지정
# 얼굴 인식 model, 랜드마크 인식기 생성
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('./content/shape_predictor_68_face_landmarks.dat')

# 양 눈 영역을 나타내는 랜드마크 인덱스
left_eye = [36, 37, 38, 39, 40, 41]
right_eye = [42, 43, 44, 45, 46, 47]
eyes = [left_eye , right_eye]

# 이진화 임계값 설정
threshold_bin = 200 

# 시선 좌표 임계값(아무방향으로의이동거리) (unit : pixel)
threshold_gaze = 5
# 마우스 민감도
sensitivity = 100

# 출력창 크기 설정
output_width, output_height = 1920, 1080

# target 좌표 초기값 :: 화면 정중앙
target_x_init, target_y_init = int(output_width/2) , int(output_height/2)

In [ ]:
# 이미지(grayscaled) 입력받아 (전처리 후) 이진화 이미지로 변환 후 반환
def get_binary_img(img):
    equalized = cv2.equalizeHist(img)
    blur = cv2.GaussianBlur(equalized, (5, 5), 0)
    # threshold = 200  # 임계값 설정
    _, binary_image = cv2.threshold(blur, threshold_bin, 255, cv2.THRESH_BINARY)

    # 객체 영역과 나머지 영역을 분할하여 결과 이미지 생성
    result = np.where(binary_image <= threshold_bin, 0, 255).astype(np.uint8)

    return result

In [53]:
# 사각형 영역에서 원(눈동자)을 검출하여 반환하는 함수
def detect_iris_in_rect(img, rect):
    # 사각형 좌표를 이용해 마스크 생성
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, np.int32([rect]), (255, 255, 255))

    # 마스크 적용
    masked = cv2.bitwise_and(img, mask)

    # 원 검출을 위한 파라미터 설정
    min_radius = 5  # 최소 원 반지름
    max_radius = 20  # 최대 원 반지름
    
    # 허프 변환을 사용하여 원 검출
    circles = cv2.HoughCircles(masked, cv2.HOUGH_GRADIENT, dp=1, minDist=50,
                               param1=50, param2=30, minRadius=min_radius, maxRadius=max_radius)
    
    # 원이 검출되지 않았거나 2개 이상의 원이 검출된 경우 None 반환
    if circles is None or len(circles[0]) != 1:
        return None
    
    return circles

#######################################################

def get_masked_img(img, rect):
    # 사각형 좌표를 이용해 마스크 생성
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, np.int32([rect]), (255, 255, 255))

    # 마스크 적용
    masked = cv2.bitwise_and(img, mask)

    return masked

def detect_iris(img):
    # 원 검출을 위한 파라미터 설정
    min_radius = 5  # 최소 원 반지름
    max_radius = 20  # 최대 원 반지름
    
    # 허프 변환을 사용하여 원 검출
    circles = cv2.HoughCircles(masked, cv2.HOUGH_GRADIENT, dp=1, minDist=50,
                               param1=50, param2=30, minRadius=min_radius, maxRadius=max_radius)
    
    # 원이 검출되지 않았거나 2개 이상의 원이 검출된 경우 None 반환
    if circles is None or len(circles[0]) != 1:
        return None
    
    return circles


In [ ]:
# 이미지에서 주어진 사각형 영역을 crop하고 거기서 눈동자를 검출하는 함수
### PENDING ###
def detect_iris_in_crop(img, rect):
    # 이미지 자르기
    x, y, w, h = rect
    cropped_img = img[y:y+h, x:x+w]

    # 자른 이미지를 이진화
    binarized_img = get_binary_img(cropped_img)
    
    # 객체 검출
    contours, _ = cv2.findContours(binarized_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    objects = []

    for contour in contours:
        # 객체의 중심과 면적 계산
        moments = cv2.moments(contour)
        if moments["m00"] == 0:
            continue
        cX = int(moments["m10"] / moments["m00"])
        cY = int(moments["m01"] / moments["m00"])
        area = cv2.contourArea(contour)

        # 객체 형태 분석
        perimeter = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.04 * perimeter, True)
        num_vertices = len(approx)

        # 원 또는 타원형 객체인지 확인
        if num_vertices >= 8:
            objectType = "Ellipse"
        else:
            objectType = "Circle"

        # 객체 정보 저장
        objects.append({"Type": objectType, "Area": area, "Center": (cX, cY)})

    return objects

In [ ]:
# 이미지 자르고 이진화하여 반환(mask XX Crop OO)
def crop_and_binarization(img, rect):
    # 이미지 자르기
    x, y, w, h = rect
    cropped_img = img[y:y+h, x:x+w]

    # 자른 이미지를 이진화 
    result = get_binary_img(cropped_img)

    return result

In [ ]:
# 이진화된 이미지의 검은색 영역 내 포함될 수 있는 최대크기의 타원을 그리고 반환하는 함수
## 조건1: 그릴 수 있는 타원의 사이즈 제한parameters (minimum_size <= 사이즈 <= maximum_size)
## 조건2: 장축/단축의 비율이 2 이상인 너무 길쭉한 타원은 제외
def find_largest_ellipse(binary_img, minimum_size, maximum_size):
    # Find contours of black regions
    contours, _ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    largest_ellipse = None
    largest_area = 0

    for contour in contours:
        # Fit ellipse to contour
        if len(contour) >= 5:
            ellipse = cv2.fitEllipse(contour)

            # Calculate area of the ellipse
            area = np.pi * ellipse[1][0] * ellipse[1][1]

            # Check if the area is within the minimum and maximum sizes
            if minimum_size <= area <= maximum_size:
                # Check if the aspect ratio is within the allowed range
                aspect_ratio = ellipse[1][0] / ellipse[1][1]
                if aspect_ratio < 2:
                    # Update the largest ellipse if the current ellipse has a larger area
                    if area > largest_area:
                        largest_ellipse = ellipse
                        largest_area = area

    return largest_ellipse

In [ ]:
def draw_ellipse(img, ellipse):
    # Create a copy of the image to draw on
    #image_with_ellipse = image.copy()

    # Convert ellipse parameters to integers
    center = tuple(map(int, ellipse[0]))
    axes = tuple(map(int, ellipse[1]))
    angle = int(ellipse[2])

    # Draw the ellipse on the image
    cv2.ellipse(img, center, axes, angle, 0, 360, (0, 255, 0), thickness=1)
    # 타원 중심점
    cv2.circle(img, center, radius=2, (0, 255, 0), thickness=-1)


In [ ]:
# circles의 정보 입력받아 이미지에 원 둘레 & 중심점을 그리는 함수
def draw_Circles(img, circles):
    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        for (x, y, r) in circles:
            cv2.circle(img, (x, y), r, (0, 255, 0), 1) # 원 둘레 그리기
            cv2.circle(img, (x, y), 2, (0, 0, 255), -1) # 중심점 그리기

In [ ]:
# pts(list)의 정보 입력받아 이미지에 사각형 그리는 함수
def draw_Rect(img, pts):
    pts_np = np.array(pts, np.int32)
    cv2.polylines(img, [pts_np], True, (0, 255, 255), 1)

In [ ]:
def init_target_point(img):
    

In [54]:
cap = cv2.VideoCapture(0)

#cv2.namedWindow('Left eye', cv2.WINDOW_NORMAL)  # 창의 크기 조정을 위해 NORMAL 모드 설정
#cv2.resizeWindow('Left eye', 30, 30)  # 원하는 크기로 조정 
#cv2.namedWindow('Right eye', cv2.WINDOW_NORMAL)  # 창의 크기 조정을 위해 NORMAL 모드 설정
#cv2.resizeWindow('Right eye', 30, 30)  # 원하는 크기로 조정 

while True:
    # 프레임 읽어오기
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)

    # 그레이스케일 후 얼굴 검출
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #equalized = cv2.equalizeHist(gray)
    #blur = cv2.GaussianBlur(equalized, (5, 5), 0)
    faces = detector(blur)

    for face in faces:
        # 얼굴의 68개 landmarks 검출 
        landmarks = predictor(gray, face)

        # 눈 영역(사각형)의 x, y 좌표를 저장할 리스트 초기화
        left_eye_pts = []
        right_eye_pts = []

        # 눈 영역을 crop할 사각 바운더리 계산
        for eye in eyes :
            mid_side_h = np.mean([landmarks.part(eye[0]).y, landmarks.part(eye[3]).y])
            up_side_dd = 2 * abs(mid_side_h - np.mean([landmarks.part(eye[1]).y, landmarks.part(eye[2]).y]))
            up_side = mid_side_h - up_side_dd
            low_side_dd = 2 * abs(mid_side_h - np.mean([landmarks.part(eye[4]).y, landmarks.part(eye[5]).y]))
            low_side = mid_side_h + low_side_dd
            left_side = landmarks.part(eye[0]).x 
            right_side = landmarks.part(eye[3]).x 
            if len(left_eye_pts) == 0 :
                left_eye_pts.append((left_side, up_side)) # 좌상단
                left_eye_pts.append((right_side, up_side)) # 우상단
                left_eye_pts.append((right_side, low_side)) # 우하단
                left_eye_pts.append((left_side, low_side)) # 좌하단
            else :
                right_eye_pts.append((left_side, up_side)) # 좌상단
                right_eye_pts.append((right_side, up_side)) # 우상단
                right_eye_pts.append((right_side, low_side)) # 우하단
                right_eye_pts.append((left_side, low_side)) # 좌하단

        # crop할 사각형 영역 그려보기 ## 수정 확인 필요 ##
        #left_eye_pts_np = np.array(left_eye_pts, np.int32)
        #right_eye_pts_np = np.array(right_eye_pts, np.int32)
        #cv2.polylines(frame, [left_eye_pts_np], True, (0, 255, 255), 1)
        #cv2.polylines(frame, [right_eye_pts_np], True, (0, 255, 255), 1)
        ##draw_Rect(frame, left_eye_pts)
        ##draw_Rect(frame, right_eye_pts)

        # L, R 안구 이미지 자르고 이진화
        left_eye_pts_np = np.array(left_eye_pts, np.int32)
        right_eye_pts_np = np.array(right_eye_pts, np.int32)
        left_eye_biimg = crop_and_binarization(frame, left_eye_pts_np)
        right_eye_biimg = crop_and_binarization(frame, right_eye_pts_np)

        # 홍채 영역 검출 후 반환
        left_iris = find_largest_ellipse(left_eye_biimg, minimum_size, maximum_size)
        right_iris = find_largest_ellipse(right_eye_biimg, minimum_size, maximum_size)

        if left_iris is not None and right_iris is not None :
            # 검출된 홍채 객체의 좌표는 crop이미지 기준 -> frame이미지 기준 좌표로 tranform
            left_center = left_iris[0]
            left_new_center = (left_center[0] + left_eye_pts_np[0][0], left_center[1] + left_eye_pts_np[0][1])
            left_iris_transformed = (left_new_center, left_iris[1], left_iris[2])
            right_center = right_iris[0]
            right_new_center = (right_center[0] + right_eye_pts_np[0][0], right_center[1] + right_eye_pts_np[0][1])
            right_iris_transformed = (right_new_center, right_iris[1], right_iris[2])
    
            # frame에 양 홍채의 경계선과 그 중심점(=시선좌표) 그리기
            draw_ellipse(frame, left_iris_transformed)
            draw_ellipse(frame, right_iris_transformed)

            # 현 frmae의 시선좌표 계산
            target_x = (left_new_center[0] + right_new_center[0]) // 2
            target_y = (left_new_center[1] + right_new_center[1]) // 2
            
            ## Gaze points vector 미검출(None)이거나 정지상태라고 판별할 경우 좌표 이동하지 않음
            ## 시선 좌표: 매 프레임 좌표 -> 벡터지향적으로 재작성
            # target좌표가 이전 frame의 좌표와 그 차이(dx,dy)가 '임계값'이하이거나 None반환일경우, 
            # target좌표는 이전 frame의 좌표를 그대로 유지(덮어쓰기)

            # 영점보정 트리거 시 -> init position of gaze in the screen (center of screen)

        
        


        


    
    #left_eye_result = detect_iris_in_rect(result, left_eye_pts)
    #right_eye_result = detect_iris_in_rect(result, right_eye_pts)
    #cv2.imshow("Left eye", left_eye_result)
    #cv2.imshow("Right eye", right_eye_result)

    # 화면에 프레임 출력
    cv2.imshow("Result", result)

    # 'q'를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 웹캠 해제 및 창 닫기
cap.release()
cv2.destroyAllWindows()